In [1]:
import os
import logging

import pandas as pd
import numpy as np
from scipy import stats

In [2]:
# Seaborn and matplotlib visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('bmh')                    
sns.set_style({'axes.grid':False}) 

# Plotly visualization
import plotly.offline as py
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected = True)

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [3]:
logging.basicConfig(format="%(asctime)s [%(name)s] [%(levelname)s] %(message)s")
logger = logging.getLogger(name="JPX")
logger.setLevel(logging.INFO)

def info(msg):
    logger.info(msg)
    
def warn(msg):
    logger.warning(msg)
    
def error(msg):
    logger.error(msg)

# Load Data

In [211]:
DATA_BASE_DIR = "./data/"

TRAIN_DIRS = [
    os.path.join(DATA_BASE_DIR, "train_files"), 
    # os.path.join(DATA_BASE_DIR, "supplemental_files")
    # Add `supplemental_files` later
]

TEST_DIR = os.path.join(DATA_BASE_DIR, "example_test_files")

info(f"train dirs: {TRAIN_DIRS}")

2022-04-26 10:29:34,010 [JPX] [INFO] train dirs: ['./data/train_files']


In [212]:
FINANCIALS_CSV_PATH = "financials.csv"
OPTIONS_CSV_PATH = "options.csv"
SECONDARY_STOCK_PRICES_CSV_PATH = "secondary_stock_prices.csv"
STOCK_PRICES_CSV_PATH = "stock_prices.csv"
TRADES_CSV_PATH = "trades.csv"

In [213]:
def read_file(dir_, filename):
    df = pd.read_csv(os.path.join(dir_, filename), parse_dates=["Date"])
    return df

train = pd.concat([read_file(dir_, STOCK_PRICES_CSV_PATH) for dir_ in TRAIN_DIRS], ignore_index=True)
test = read_file(TEST_DIR, STOCK_PRICES_CSV_PATH)

## AdjustmentFactor
Adjust prices + volume based on `AdjustmentFactor`

TODO: Assuming that the data is sorted based on time


In [214]:
train["CumulativeAdjustmentFactor"] = train.groupby("SecuritiesCode")["AdjustmentFactor"].apply(
    lambda x: x.shift(1).fillna(1.0).cumprod()
)
train["Close"] = train["Close"] / train["CumulativeAdjustmentFactor"]
train["High"] = train["High"] / train["CumulativeAdjustmentFactor"]
train["Low"] = train["Low"] / train["CumulativeAdjustmentFactor"]
train["Open"] = train["Open"] / train["CumulativeAdjustmentFactor"]
train["Volume"] = train["Volume"] * train["CumulativeAdjustmentFactor"]

In [215]:
train.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,CumulativeAdjustmentFactor
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400.0,1.0,NaN,False,0.000730,1.0
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500.0,1.0,NaN,False,0.012324,1.0
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800.0,1.0,NaN,False,0.006154,1.0
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300.0,1.0,NaN,False,0.011053,1.0
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800.0,1.0,NaN,False,0.003026,1.0


In [216]:
train[train["SecuritiesCode"] == 2768][["Close", "AdjustmentFactor"]].iloc[-50:-43]

,Close,AdjustmentFactor
2232789,347.0,1.0
2234789,358.0,1.0
2236789,359.0,1.0
2238789,367.0,5.0
2240789,357.6,1.0
2242789,368.0,1.0
2244789,349.0,1.0


In [217]:
train[train["SecuritiesCode"] == 1712][["Close", "AdjustmentFactor", "CumulativeAdjustmentFactor"]].iloc[33:38]

,Close,AdjustmentFactor,CumulativeAdjustmentFactor
61563,1592.0,1.000000,1.000000
63428,1587.0,1.000000,1.000000
65293,1581.0,0.833333,1.000000
67159,1551.6,1.000000,0.833333
69025,1579.2,1.000000,0.833333


In [218]:
# For test data
test["CumulativeAdjustmentFactor"] = test.groupby("SecuritiesCode")["AdjustmentFactor"].apply(
    lambda x: x.shift(1).fillna(1.0).cumprod()
)
test["Close"] = test["Close"] / test["CumulativeAdjustmentFactor"]
test["High"] = test["High"] / test["CumulativeAdjustmentFactor"]
test["Low"] = test["Low"] / test["CumulativeAdjustmentFactor"]
test["Open"] = test["Open"] / test["CumulativeAdjustmentFactor"]
test["Volume"] = test["Volume"] * test["CumulativeAdjustmentFactor"]

## Dividend based adjustments


In [219]:
train[train["ExpectedDividend"] > 0].tail()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,CumulativeAdjustmentFactor
2320123,20211125_8155,2021-11-25,8155,2475.0,2484.0,2437.0,2444.0,50500.0,1.0,19.0,False,-0.026338,1.0
2320290,20211125_8923,2021-11-25,8923,1047.0,1051.0,1030.0,1030.0,389700.0,1.0,38.0,False,-0.073099,1.0
2320370,20211125_9369,2021-11-25,9369,2000.0,2009.0,1989.0,1990.0,69100.0,1.0,23.0,False,-0.057692,1.0
2320456,20211125_9717,2021-11-25,9717,1258.0,1260.0,1222.0,1223.0,313200.0,1.0,50.0,False,-0.127517,1.0
2320489,20211125_9837,2021-11-25,9837,671.0,677.0,671.0,672.0,12100.0,1.0,11.0,False,-0.043675,1.0


In [220]:
# train[train["SecuritiesCode"] == 3421][["Open", "Close", "ExpectedDividend", "Date"]].loc[16500:36500]
# train[train["SecuritiesCode"] == 1928][["Open", "Close", "ExpectedDividend", "Date"]].loc[16500:36500]

In [221]:
# train[train["SecuritiesCode"] == 3421][["Open", "Close", "ExpectedDividend", "Date"]].loc[16500:36500]
# train[train["SecuritiesCode"] == 9717][["Open", "Close", "ExpectedDividend", "Date"]].loc[2316456:2329456]

In [222]:
train["ExpectedDividendFactor"] = train.groupby("SecuritiesCode")["ExpectedDividend"].apply(
    lambda x: x.shift(2).fillna(0).cumsum() # Shift by for ex-dividend date
)
train["CloseDividentAdj"] = train["Close"] + train["ExpectedDividendFactor"]
train["HighDividentAdj"] = train["High"] + train["CumulativeAdjustmentFactor"]
train["LowDividentAdj"] = train["Low"] + train["CumulativeAdjustmentFactor"]
train["OpenDividentAdj"] = train["Open"] + train["CumulativeAdjustmentFactor"]

In [223]:
# train[train["SecuritiesCode"] == 1928][["CloseAdj", "Close", "ExpectedDividend", "ExpectedDividendFactor", "Date"]].loc[16500:36500]
# train[train["SecuritiesCode"] == 1928][["Open", "Close", "ExpectedDividend", "ExpectedDividendFactor", "Date"]].loc[16500:36500]

In [224]:
warn(train[train["CloseDividentAdj"] < 0].shape)
info(train[train["CloseDividentAdj"] >= 0].shape)

2022-04-26 10:29:38,149 [JPX] [WARNING] (0, 18)
2022-04-26 10:29:38,245 [JPX] [INFO] (2324923, 18)


In [225]:
train[train["SecuritiesCode"] == 9717][["Close", "CloseDividentAdj"]].iplot()

In [226]:
train.columns

Index(['RowId', 'Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close',
       'Volume', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag',
       'Target', 'CumulativeAdjustmentFactor', 'ExpectedDividendFactor',
       'CloseDividentAdj', 'HighDividentAdj', 'LowDividentAdj',
       'OpenDividentAdj'],
      dtype='object')

In [227]:
train.shape

(2332531, 18)

In [228]:
# For test data
test["ExpectedDividendFactor"] = test.groupby("SecuritiesCode")["ExpectedDividend"].apply(
    lambda x: x.shift(2).fillna(0).cumsum() # Shift by for ex-dividend date
)
test["CloseDividentAdj"] = test["Close"] + test["ExpectedDividendFactor"]
test["HighDividentAdj"] = test["High"] + test["CumulativeAdjustmentFactor"]
test["LowDividentAdj"] = test["Low"] + test["CumulativeAdjustmentFactor"]
test["OpenDividentAdj"] = test["Open"] + test["CumulativeAdjustmentFactor"]

# Financials

In [229]:
# Read data types from spec
spec_df = pd.read_csv("./data/data_specifications/stock_fin_spec.csv", index_col="Column")
spec_df.head()

,Sample value,Type,Addendum,Remarks
Column,,,,
DisclosureNumber,20161025419878,Int64,NaN,Unique ID for disclosure documents.
DateCode,20170106_7888,string,NaN,combination of TradeDate and LocalCode (this i...
Date,2017-01-06 0:00:00,date,NaN,Trade date. This column is used to align with ...
SecuritiesCode,7888,Int64,NaN,Local Securities Code
DisclosedDate,2017-01-06 0:00:00,date,NaN,Date on which the document disclosed.


In [230]:
spec_df["Type"].unique()

array(['Int64', 'string', 'date', 'time', 'float', 'boolean'],
      dtype=object)

In [231]:
fin_columns_datetypes_list

['Date',
 'DisclosedDate',
 'DisclosedTime',
 'CurrentPeriodEndDate',
 'TypeOfCurrentPeriod',
 'CurrentFiscalYearStartDate',
 'CurrentFiscalYearEndDate']

In [232]:
spec_df["PDType"] = spec_df["Type"]
spec_df["PDType"][spec_df["Type"] == 'date'] = "datetime64"
spec_df["PDType"][spec_df["Type"] == 'time'] = "datetime64"
spec_df["PDType"][spec_df["Type"] == 'Int64'] = "float64"

In [233]:
fin_columns_types = spec_df[spec_df["PDType"] != 'datetime64']["PDType"].to_dict()
fin_columns_types_int64 = spec_df[spec_df["Type"] == 'Int64']["Type"].to_dict() # Using Type col
fin_columns_datetypes_list = list(spec_df[spec_df["PDType"] == 'datetime64']["PDType"].to_dict().keys())

In [234]:
def read_financials_file(dir_, filename):
    fin_df = pd.read_csv(os.path.join(dir_, filename), 
                         dtype=fin_columns_types, 
                         parse_dates=fin_columns_datetypes_list,
                         na_values=["－", pd.NA])
    fin_df = fin_df.astype(fin_columns_types_int64)
    # Since same column is called by `RowId` in close price dataframe
    fin_df.rename(columns={"DateCode": "RowId"}, inplace=True)
    return fin_df

train_financials = pd.concat(
    [read_financials_file(dir_, FINANCIALS_CSV_PATH) for dir_ in TRAIN_DIRS], 
    ignore_index=True)
test_financials = read_financials_file(TEST_DIR, FINANCIALS_CSV_PATH)

In [235]:
info(train_financials.shape)
# Currently using only limited types of documents from financials
# Use more types of documents TODO?
doc_types = [
    "FYFinancialStatements_Consolidated_JP",
    "1QFinancialStatements_Consolidated_JP",
    "2QFinancialStatements_Consolidated_JP",
    "3QFinancialStatements_Consolidated_JP"
]
train_financials = train_financials[train_financials["TypeOfDocument"].isin(doc_types)].reset_index()

info(train_financials.shape)

# Sort based on `CurrentPeriodEndDate` so that most recent account records data comes towards the top
train_financials.sort_values(by="CurrentPeriodEndDate", inplace=True, ascending=False)
train_financials = train_financials.groupby("RowId").first().reset_index()
info(train_financials.shape)

2022-04-26 10:29:39,213 [JPX] [INFO] (92956, 45)
2022-04-26 10:29:39,237 [JPX] [INFO] (58850, 46)
2022-04-26 10:29:39,382 [JPX] [INFO] (58840, 46)


In [236]:
# Same for test data

info(test_financials.shape)
test_financials = test_financials[test_financials["TypeOfDocument"].isin(doc_types)].reset_index()

info(test_financials.shape)

# Sort based on `CurrentPeriodEndDate` so that most recent account records data comes towards the top
test_financials.sort_values(by="CurrentPeriodEndDate", inplace=True, ascending=False)
test_financials = test_financials.groupby("RowId").first().reset_index()
info(test_financials.shape)

2022-04-26 10:29:39,386 [JPX] [INFO] (14, 45)
2022-04-26 10:29:39,390 [JPX] [INFO] (7, 46)
2022-04-26 10:29:39,399 [JPX] [INFO] (7, 46)


In [237]:
train.columns

Index(['RowId', 'Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close',
       'Volume', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag',
       'Target', 'CumulativeAdjustmentFactor', 'ExpectedDividendFactor',
       'CloseDividentAdj', 'HighDividentAdj', 'LowDividentAdj',
       'OpenDividentAdj'],
      dtype='object')

In [238]:
info(train.shape)
train = pd.merge(train, train_financials, how="left", on=["RowId", "SecuritiesCode", "Date"])
info(train.shape)

2022-04-26 10:29:39,407 [JPX] [INFO] (2332531, 18)
2022-04-26 10:29:40,986 [JPX] [INFO] (2332531, 61)


In [239]:
# Same for test data
info(test.shape)
test = pd.merge(test, test_financials, how="left", on=["RowId", "SecuritiesCode", "Date"])
info(test.shape)

2022-04-26 10:29:40,990 [JPX] [INFO] (4000, 17)
2022-04-26 10:29:41,001 [JPX] [INFO] (4000, 60)


In [245]:
# Drop some columns which are not required / can be merged
train.drop(["RowId", "index", "DisclosureNumber"], axis=1, inplace=True)
test.drop(["RowId", "index", "DisclosureNumber"], axis=1, inplace=True)

In [247]:
info(train.shape)
info(test.shape) # Target is missing here

2022-04-26 10:33:38,020 [JPX] [INFO] (2332531, 58)
2022-04-26 10:33:38,023 [JPX] [INFO] (4000, 57)


In [249]:
train.dtypes

Date                                                                            datetime64[ns]
SecuritiesCode                                                                           int64
Open                                                                                   float64
High                                                                                   float64
Low                                                                                    float64
Close                                                                                  float64
Volume                                                                                 float64
AdjustmentFactor                                                                       float64
ExpectedDividend                                                                       float64
SupervisionFlag                                                                           bool
Target                                            

In [ ]:
# Clean datetimes + remove more cols based on coorelation
# TODO